# Object Detection on CRAL


Tutorial on how to use CRAL library for object detection using RetinaNet and YOLOv3.

Backbones supported for RetinaNet: `ResNet50` `ResNet101` `ResNet152`

Backbones supported for YOLOv3: `DarkNet53`


### 0. **Getting started** 



0.1 Install CRAL

In [ ]:
!pip3 install cral

0.2 Create an account on [track.segmind.com](https://track.segmind.com/) to track runs

0.3 Configure CRAL


CRAL needs to be configured with `cral config` after installing the library. After running this, you will be asked for the email and password.
Only after this will the tracking of the runs be possible.

In [ ]:
!cral config

### 1. **Experiment tracking** 


This is a mandatory step to be run before training. It provides you statistics on training data and logs all the metrics, losses etc. in an online dashboard making it easier to evaluate the model and compare between different runs, which can help you set the right hyperparameters. 

Generate experiment ID on [track.segmind.com](https://track.segmind.com/) to track your run and refer [here](https://cral.segmind.com/en/api/tracking) for instructions to log custom metadata and artifacts.

In [ ]:
from cral.tracking import set_experiment

set_experiment("your-experiment-ID")

###2. **Initiate object detection pipeline**


This initiates the classification pipeline. It also creates project related files that are needed later during training and inference.

In [ ]:
from cral.pipeline import ObjectDetectionPipe

pipe= ObjectDetectionPipe()

###3. **Add data**


This parses the dataset once to generate metadata and versions the data. 

Change **supported_annotation_format** to one of the supported formats: `yolo` `coco` `pascal_voc`.

If you do not have a seperate validation set, you can divide the training dataset into training and validation using *split* argument. Otherwise, provide the path to the validation images and annotations.

In [ ]:
pipe.add_data(
    train_images_dir= "/path/to/images/folder",
    train_anno_dir= "/path/to/annotation/folder",
    annotation_format= "supported_annotation_format",
    val_images_dir= None,
    val_anno_dir= None,
    split= 0.2)                                             

###4. **Lock data**

This parses the data and makes Tf-records. Tf-records are part of the [tf.data](https://www.tensorflow.org/guide/data) API which makes the data ingestion into the model faster and hence reduces the training time.



In [ ]:
pipe.lock_data()

###5. **Set algorithm**

You can instantiate different object detection models here.


###5.1. For RetinaNet


You can use any of the supported backbones for RetinaNet. Do not forget to change **backbone_name** to the backbone of your choice. 


You can find the list of supported backbones and accepted arguments [here](https://cral.segmind.com/api/models/ObjectDetection#retinanet). Also, take a look at how the hyperparamets work and how you can change them [here](https://blog.segmind.com/an-introduction-to-retinanet-and-how-we-make-it-easier-to-use/).

In [ ]:
from cral.models.object_detection import RetinanetConfig

retinanet_config= RetinanetConfig()

pipe.set_algo(
    feature_extractor= "backbone_name",  
    config= retinanet_config)

###5.2. For YOLOv3


Currently CRAL supports DarkNet53 as a backbone for YOLOv3.


You can find the list of accepted arguments for configuring your YOLOv3 model [here](https://cral.segmind.com/api/models/ObjectDetection#yolov3).

In [ ]:
from cral.models.object_detection import Yolov3Config

yolo_config= Yolov3Config()

pipe.set_algo(
    feature_extractor= "darknet-53",
    config= yolo_config)

###6. **Train**

Now that you have set the model, you can start training.
Using train is very similar to the *fit* function in tensorflow.keras and it accepts many of the same arguments like epochs, batch_size etc. All your loss and metrics will be streamed from here onto [track.segmind.com](http://track.segmind.com/) automatically where you can see and evaluate your model easily.

In [ ]:
pipe.train(
    num_epochs= 20,
    batch_size= 4,
    snapshot_prefix= "test",
    snapshot_path= "snapshots",
    snapshot_every_n= 2)

###7. **Prediction**

After training you can use the model to get predictions on images that maybe or may not be a part of your initial dataset.

Calling `prediction_model` will give you a function that can be used to get predictions on images.

In [ ]:
checkpoint_file= "snapshots/test_final"
prediction_func= pipe.prediction_model(checkpoint_file= checkpoint_file)

img_path= "path/to/test/image"

###7.1. For RetinaNet

In [ ]:
from cral.models.object_detection import annotate_image
from PIL import Image
import numpy as np

# make prediction
bboxes, scores, labels= prediction_func(img_path)

# draw predictions on image and save
image_array= np.array(Image.open(img_path))
drawn_image= annotate_image(image_array, bboxes, scores, labels, threshold=0.5)
drawn_image.save("image.jpeg")

###7.2. For YOLOv3

In [ ]:
from cral.models.object_detection.yolov3_2.utils import annotate_image

# make prediction
orignal_image, predictions= prediction_func(img_path)

# draw predictions on image and save
annotate_image(orignal_image, predictions, save_path="image.jpeg")